### Importamos librerías

In [1]:
import os
import tweepy as tw
import pandas as pd
import re
import numpy as np
from datetime import datetime
from tqdm import tqdm
import openpyxl

### Credenciales para el uso de la API


In [2]:
consumer_key= 'qLBuvGr2AzL5TmARGF2vIYLar'
consumer_secret= 'WwPMJxsf7ofyYLsqEr1Tj8dy636eT9D5Jmgp4ab2JgjT5RpQJo'
access_token= '1267192889416847362-4WE0cPAp56rWJ7sGedH909p8QC4P0v'
access_token_secret= 'ByZnnbxuiRoBNbfoWJTCj6xiXuwCwoLorwVOOfZPD44ln'

In [3]:
auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)

### Funcion que genere fechas-horas aleatorias 
- El rango será entre el 1 de enero de 2020 y el 15 de julio de 2021
- Se generará una fecha y esta será el parámetro *toDate*
- Se aleatorizará de la siguiente manera:
    · año -> 2020 o 2021\
    · mes:\
        >>> si año = 2020 -> aleatorio de 1 a 12\
        >>> si año = 2021 -> aleatorio de 1 a 7\
    · dia:\
           >>>  si mes = 1 / 3 / 5 / 7 / 8 / 10 / 12 -> aleatorio de 1 a 31\
           >>>  si mes = 4 / 6 / 9 / 11 -> aleatorio de 1 a 30\
           >>>  si mes = 2 -> aleatorio de 1 a 28\
    · hora -> de 0 a 24\
    · minutos -> constante a 00

In [5]:
def generate_toDate():

    # Generar año
    year =  str(np.random.choice([2020, 2021]))

    # Generar mes
    if year == '2020':
        month = str(np.random.randint(low=1, high=12))
    else:
        month = str(np.random.randint(low=1, high=7))
    if len(month) == 1: month = '0' + month
    
    # Generar dia
    if month in ['01', '03', '05', '07', '08', '10', '12']:
        day = str(np.random.randint(low=1, high=31))
    elif month in ['04', '06', '09', '11']:
        day = str(np.random.randint(low=1, high=30))
    else:
        day = str(np.random.randint(low=1, high=28))
    if len(day) == 1: day = '0' + day

    # Generar hora
    hour = str(np.random.randint(low=0, high=24))
    if len(hour) == 1: hour = '0' + hour

    
    toDate = year + month + day + hour + '00'

    return toDate

## EXTRACCION FULL ARCHIVE

In [6]:
query_word = 'telefonica OR movistar'
until = '202103281200'

In [129]:
x = api.search_full_archive(environment_name='fullarchive', query=query_word,
                            toDate=until, maxResults=100)

In [20]:
def tweets_to_DF(tweets):
    
    df=pd.DataFrame(columns=['tweet_id', 'text', 'created_at', 'retweeted', 'retweet_count', 'favorite_count','user_ verified',
                             'user_id', 'user_name', 'user_location', 'geo', 'coordinates', 'place', 'user_notificacion', 
                             'user_followers', 'user_friends','user_withheld_in_countries', 'is_reply', 'finished_tweet',
                             'status_count'])    
    i=0
    for tweet in tweets:

        new_row = {
        'tweet_id': tweet._json['id'],
        'text': tweet._json['text'],
        'created_at': tweet._json['created_at'],
        'retweeted': tweet._json['retweeted'],
        'retweet_count': tweet._json['retweet_count'],
        'favorite_count': tweet._json['favorite_count'],
        'user_verified': tweet._json['user']['verified'], 
        'user_id': tweet._json['user']['id'],
        'user_name': tweet._json['user']['screen_name'],
        'user_location': tweet._json['user']['location'],
        'geo': tweet._json['geo'],
        'coordinates': tweet._json['coordinates'],
        'place': tweet._json['place'],
        'user_notificacion': tweet._json['user']['notifications'],
        'user_followers': tweet._json['user']['followers_count'],
        'user_friends': tweet._json['user']['friends_count'],
        'user_withheld_in_countries':  tweet._json['user']['withheld_in_countries'],
        'is_reply': tweet._json['in_reply_to_status_id'],
        'finished_tweet': tweet._json['truncated'],
        'status_count':tweet._json['user']['statuses_count']
        }
        
        df.loc[i] = new_row # Añadimos la info al dataframe en una nueva fila
        i = i + 1

    return df


# IMPORTANTE!
Cada vez que se haga una extracción, crear un nuevo dataframe y guardarlo en excel de la siguiente manera: \
\
    *df_fa_x = tweets_to_DF(x)* \
    *df_fa_x.to_excel('extracciones/df_fa_x.xlsx')* \
    \
**Sustituir 'x' por un número siguiendo una numeración consecutiva**

In [71]:
df_fa_1 = tweets_to_DF(x)
df_fa_1.to_excel('extracciones/df_fa_1.xlsx')

Combinar DataFrames con las extracciones de prueba. Con esto se conforma el dataframe al que se le irán añadiendo las futuras extracciones:

In [14]:
df_fa_1 = pd.read_excel('extracciones/df_fa_1.xlsx')
df_fa_2 = pd.read_excel('extracciones/df_fa_2.xlsx')
df_fa_3 = pd.read_excel('extracciones/df_fa_3.xlsx')
df_fa_4 = pd.read_excel('extracciones/df_fa_4.xlsx')

In [17]:
#NO VOLVER A EJECUTAR
df_fa = df_fa_1.append(df_fa_2, ignore_index=True)
df_fa = df_fa.append(df_fa_3, ignore_index=True)
df_fa = df_fa.append(df_fa_4, ignore_index=True)

### Bucle para automatizar la extraccion con las siguientes operaciones:
- Obtener *toDate* aleatoria
- Ejecutar *api.search_full_archive*
- Generar dataframe a partir de los tweets extraidos
- Guardar extraccion en excel
- Añadir extraccion a DataFrame conjunto *df_fa*

## IMPORTANTE: Por seguridad, al acabar un bucle, guardamos *df_fa* en excel

# IMPORTANTISIMO: Antes de ejecutar el bucle, asegurarse que df_fa está cargado con todas las extracciones y revisar que la variable n que se utiliza en **'file_name'** es el correcto para que las extracciones se vayan guardando en excel con numeracion consecutiva

In [31]:
for i in range(6):
    
    # Obtener toDate aleatoria
    to_date = generate_toDate()

    # Ejecutar api.search_full_archive
    x = api.search_full_archive(environment_name='fullarchive', query='telefonica OR movistar',
                            toDate=to_date, maxResults=100)
                            
    # Generar dataframe a partir de los tweets extraidos
    df_fa_n = tweets_to_DF(x)

    # Guardar extraccion en excel #ACTUALIZAR NUM 4 POR NUMERO DE ARCHIVOS YA EXTRAIDOS!!!!!
    file_name = 'extracciones/df_fa_' + str(i+1+4) + '.xlsx'
    df_fa_n.to_excel(file_name)

    # Añadir extraccion al DataFrame conjunto 'df_fa'
    df_fa = df_fa.append(df_fa_n, ignore_index=True)

    


Guardado de el df conjunto por seguridad

In [33]:
df_fa.to_excel('extracciones/df_fa_v1000.xlsx')